# Import necessary packages

In [51]:
import os
import sys
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import PurePath
import warnings
warnings.filterwarnings("ignore")

# Set inputs for plot module

In [52]:
# threshold = 1  # use match results computed by 1 times radius
threshold = 2    # use match results computed by 2 times radius
# threshold = 4  # use match results computed by 4 times radius  

# ignore_duration = 30    # use match results computed by ignore events which duration is less than 30 seconds
ignore_duration = 60  # use match results computed by ignore events which duration is less than 60 seconds
# ignore_duration = 120 # use match results computed by ignore events which duration is less than 120 seconds

short_stack = 0 # use match results computed by short stack
# short_stack = 1 # use match results computed by all experimental dates

print("Input parameters:")
print("threshold = ", threshold)
print("ignore_duration = ", ignore_duration)
print("short_stack = ", short_stack)

Input parameters:
threshold =  2
ignore_duration =  60
short_stack =  0


# Load Data

The module's data consists of Stanford Defined Events, Team Defined Events, and their respective classifications.

In [53]:
def get_total_events(events_PN, is_true_event=False):
    """
    Get the toal events from the match results, including positive, negative and D events.
    Args:
        events_PN: the events with positive and negative classes
    Returns:
        the events with positive, negative and D classes
    """
    start_column = "EventStart"
    end_column = "EventEnd"
    class_column = "Eventclass"

    events_date = events_PN[start_column].apply(lambda x: x.date())
    events_PN_aNA = pd.DataFrame(columns=events_PN.columns)

    for date in events_date.unique():
        tmp_df = events_PN.loc[events_date == date, :]
        tmp_df = tmp_df.sort_values(by=[start_column, end_column]).reset_index(drop=True)
        if tmp_df.shape[0] == 0:
            continue
        else:
            if tmp_df.loc[0, start_column] > pd.to_datetime(date):
                m = events_PN_aNA.shape[0]
                events_PN_aNA.loc[m, start_column] = pd.to_datetime(date)
                events_PN_aNA.loc[m, end_column] = tmp_df.loc[0, start_column] - datetime.timedelta(seconds=1)
                events_PN_aNA.loc[m, class_column] = str("NA")
                events_PN_aNA.loc[m, "label"] = str("NA")
            for i in range(1, tmp_df.shape[0]):
                if tmp_df.loc[i, start_column] - tmp_df.loc[i-1, end_column] > datetime.timedelta(seconds=1):
                    m = events_PN_aNA.shape[0]
                    events_PN_aNA.loc[m, start_column] = tmp_df.loc[i-1, end_column] + datetime.timedelta(seconds=1)
                    events_PN_aNA.loc[m, end_column] = tmp_df.loc[i, start_column] - datetime.timedelta(seconds=1)
                    events_PN_aNA.loc[m, class_column] = str("NA")
                    events_PN_aNA.loc[m, "label"] = str("NA")

            if tmp_df.loc[tmp_df.shape[0]-1, end_column] < pd.to_datetime(date) + datetime.timedelta(days=1) - datetime.timedelta(seconds=1):
                m = events_PN_aNA.shape[0]
                events_PN_aNA.loc[m, start_column] = tmp_df.loc[tmp_df.shape[0]-1, end_column] + datetime.timedelta(seconds=1)
                events_PN_aNA.loc[m, end_column] = pd.to_datetime(date) + datetime.timedelta(days=1) - datetime.timedelta(seconds=1)
                events_PN_aNA.loc[m, class_column] = str("NA")
                events_PN_aNA.loc[m, "label"] = str("NA")
    
    # combine the events_PN and events_PN_aNA
    events_PN = pd.concat([events_PN, events_PN_aNA], axis=0).reset_index(drop=True)

    # sort the events_PN by start time
    events_PN = events_PN.sort_values(by=[start_column, end_column]).reset_index(drop=True)

    return events_PN


def get_events(root_path, sensor_name, threshold, ignore_duration, short_stack):

    if short_stack:
        path_pre_string = "ss_"
    else:
        path_pre_string = ""

    # construct the path of the match results
    match_path = PurePath(root_path, 
                              path_pre_string + "{}xradius_{}xseconds_{}_match_events.csv".format(threshold, ignore_duration, sensor_name))
    
    # read the match results
    match_data = pd.read_csv(match_path)
    match_data["EventStart"] = pd.to_datetime(match_data["EventStart"])
    match_data["EventEnd"] = pd.to_datetime(match_data["EventEnd"])

    return match_data

def get_timebased_events(root_path, sensor_name, threshold, ignore_duration, short_stack):

    if short_stack:
        path_pre_string = "ss_"
    else:
        path_pre_string = ""

    # construct the path of the match results
    match_path = PurePath(root_path, 
                              path_pre_string + "{}xradius_{}xseconds_{}_match_events.csv".format(threshold, ignore_duration, sensor_name))
    # read the match results
    match_data = pd.read_csv(match_path)
    match_data["Datetime (UTC)"] = pd.to_datetime(match_data["Datetime (UTC)"])
    match_data["EmissionDate"] = match_data["Datetime (UTC)"].apply(lambda x: pd.to_datetime(x).date())
    match_data = match_data.fillna(str("NA"))

    return match_data

# set the root path of the match results
stanford_root_path = PurePath("../../results/03_DetectionAnalysis/Test-case Matching Data/Stanford Defined Events")
team_root_path = PurePath("../../results/03_DetectionAnalysis/Test-case Matching Data/Team Defined Events")
time_root_path = PurePath("../../results/03_DetectionAnalysis/Test-case Matching Data/Time-based Events")

# set the sensor names
sensor_names = ["Andium", "Canary", "Ecoteco", "Kuva", "Oiler", "Qube", "Sensirion"]

# get the match results
stanford_events_dict = {}
team_events_dict = {}
time_events_dict = {}
for sensor_name in sensor_names:
    print("Processing {}...".format(sensor_name))
    stanford_events_dict[sensor_name] = get_events(stanford_root_path, sensor_name, threshold, ignore_duration, short_stack)
    stanford_events_dict[sensor_name] = get_total_events(stanford_events_dict[sensor_name], is_true_event=True)
    print("\tStanford events: {}".format(stanford_events_dict[sensor_name].shape[0]))
    team_events_dict[sensor_name] = get_events(team_root_path, sensor_name, threshold, ignore_duration, short_stack)
    team_events_dict[sensor_name] = get_total_events(team_events_dict[sensor_name], is_true_event=False)
    print("\tTeam events: {}".format(team_events_dict[sensor_name].shape[0]))
    # specially, load Time-based match results
    time_events_dict[sensor_name] = get_timebased_events(time_root_path, sensor_name, threshold, ignore_duration, short_stack)
    print("\tTime-based events: {}".format(time_events_dict[sensor_name].shape[0]))

Processing Andium...
	Stanford events: 574
	Team events: 244
	Time-based events: 4492800
Processing Canary...
	Stanford events: 112
	Team events: 38
	Time-based events: 4492800
Processing Ecoteco...
	Stanford events: 230
	Team events: 1050
	Time-based events: 4492800
Processing Kuva...
	Stanford events: 503
	Team events: 393
	Time-based events: 4492800
Processing Oiler...
	Stanford events: 305
	Team events: 263
	Time-based events: 4492800
Processing Qube...
	Stanford events: 298
	Team events: 218
	Time-based events: 4492800
Processing Sensirion...
	Stanford events: 321
	Team events: 118
	Time-based events: 4492800


# Plot charts

In [54]:
plt.rc('font', size=20)


def plot_comparison_figuires(true_event_dfs, report_event_dfs, time_event_dfs, plot_sensors, save_dir):
    """
    Rendering a comparative visualization,
    Parameters:
        true_event_dfs: Actual events
        report_event_dfs: Reported events
        time_event_dfs: Time-based events
        plot_sensors: Sensor names
        save_dir: Save directory
    """

    for sensor_name in plot_sensors:
        true_event_df = true_event_dfs[sensor_name]
        report_event_df = report_event_dfs[sensor_name]        
        true_event_df = true_event_df.sort_values(by="EventStart")
        report_event_df = report_event_df.sort_values(by="EventStart")
        time_event_df = time_event_dfs[sensor_name]

        # load the true events
        true_event_df["is_P"] = np.nan
        true_event_df.loc[true_event_df["EventClass"] == "P", "is_P"] = 1
        true_event_df.loc[true_event_df["EventClass"] == "N", "is_P"] = 0
        true_event_df["EventStart"] = true_event_df["EventStart"].apply(lambda x: pd.to_datetime(x))
        true_event_df["EventEnd"] = true_event_df["EventEnd"].apply(lambda x: pd.to_datetime(x))
        true_event_df["EventDate"] = ""
        true_event_df["EventDate"] = true_event_df["EventStart"].apply(lambda x: x.date())

        # load the report events
        report_event_df["is_P"] = np.nan
        report_event_df.loc[report_event_df["EventClass"] == "P", "is_P"] = 1
        report_event_df.loc[report_event_df["EventClass"] == "N", "is_P"] = 0
        report_event_df["EventStart"] = report_event_df["EventStart"].apply(lambda x: pd.to_datetime(x))
        report_event_df["EventEnd"] = report_event_df["EventEnd"].apply(lambda x: pd.to_datetime(x))
        report_event_df["EventDate"] = ""
        report_event_df["EventDate"] = report_event_df["EventStart"].apply(lambda x: x.date())


        # from 2022-10-10 to 2022-11-30, find the true events and report events of each day
        start_date = pd.to_datetime("2022-10-10")
        end_date = pd.to_datetime("2022-11-30")
        date_range = pd.date_range(start_date, end_date, freq="D")
        
        # save the plot
        plot_save_dir = PurePath(save_dir, sensor_name)
        if not os.path.exists(plot_save_dir):
            os.makedirs(plot_save_dir)

        with tqdm(total=len(date_range)) as pbar:
            for date in date_range:
                print(date, sensor_name)
                date = pd.to_datetime(date)
                pbar.set_description(f"{sensor_name:<10}")
                pbar.update(1)
                tmp_true_event_df = true_event_df[true_event_df["EventDate"] == date.date()].copy().reset_index(drop=True)
                tmp_report_event_df = report_event_df[report_event_df["EventDate"] == date.date()].copy().reset_index(drop=True)
                tmp_time_event_df = time_event_df[time_event_df["EmissionDate"] == date.date()].copy().reset_index(drop=True)
                print((tmp_time_event_df["norm_class"] != "NA").sum())
                print(tmp_true_event_df.shape[0])
                print(tmp_report_event_df.shape[0])


                if tmp_true_event_df.shape[0] == 0 or tmp_report_event_df.shape[0] == 0:
                    continue

                # extract the start and end time of the true events and report events, mainly for fine-tuning the x-axis
                # the start and end time is the minimum and maximum time of the true events which are labeled as "P"
                if tmp_true_event_df.shape[0] == 1:
                    true_start = tmp_true_event_df["EventStart"].values[0]
                    true_end = tmp_true_event_df["EventEnd"].values[0]
                else:
                    true_start = tmp_true_event_df["EventEnd"].values[0]
                    true_end = tmp_true_event_df["EventStart"].values[-1]

                # the start and end time is the minimum and maximum time of the report events which are labeled as "P"
                if report_event_df.shape[0] == 1:
                    report_start = tmp_report_event_df["EventStart"].values[0]
                    report_end = tmp_report_event_df["EventEnd"].values[0]
                else:
                    report_start = tmp_report_event_df["EventEnd"].values[0]
                    report_end = tmp_report_event_df["EventStart"].values[-1]

                # set the start(end) time to the minimum(maximum) time of the true start(end) time and report start(end) time
                origin_start = min(true_start, report_start)
                origin_end = max(true_end, report_end)

                # shift the start and end time to make the plot more beautiful
                shift_time = (origin_end - origin_start) / pd.Timedelta(seconds=1)
                end = min(origin_end + pd.Timedelta(seconds=shift_time * 0.1), date + pd.Timedelta(days=1) - pd.Timedelta(seconds=1))
                start = max(origin_start - pd.Timedelta(seconds=shift_time * 0.1), date)
                # start = pd.to_datetime(date)
                # end = start + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)

                # compute the continuous true events and report events
                continuous_true_event_df = pd.DataFrame(columns=["DateTime (UTC)", "is_P"])
                continuous_report_event_df = pd.DataFrame(columns=["DateTime (UTC)", "is_P"])

                # process the true events
                continuous_true_event_df["DateTime (UTC)"] = pd.date_range(start, end, freq="S").map(lambda x: pd.to_datetime(x))
                continuous_true_event_df["is_P"] = np.nan
                for i in range(tmp_true_event_df.shape[0]):
                    cur_event_start = tmp_true_event_df["EventStart"].values[i]
                    cur_event_end = tmp_true_event_df["EventEnd"].values[i]
                    label = tmp_true_event_df["EventClass"].values[i]

                    if label == "P":
                        continuous_true_event_df.loc[continuous_true_event_df["DateTime (UTC)"].between(cur_event_start - pd.Timedelta(seconds=1), cur_event_end+pd.Timedelta(seconds=1), inclusive="both"), "is_P"] = 1
                    elif label == "N":
                        continuous_true_event_df.loc[continuous_true_event_df["DateTime (UTC)"].between(cur_event_start - pd.Timedelta(seconds=1), cur_event_end+pd.Timedelta(seconds=1), inclusive="both"), "is_P"] = 0
                
                # process the report events, make the report events continuous, and the value of "is_P" is 1 if the report event is labeled as "P", otherwise 0
                continuous_report_event_df["DateTime (UTC)"] = pd.date_range(start, end, freq="S").map(lambda x: pd.to_datetime(x))
                continuous_report_event_df["is_P"] = np.nan
                for i in range(tmp_report_event_df.shape[0]):
                    cur_event_start = tmp_report_event_df["EventStart"].values[i]
                    cur_event_end = tmp_report_event_df["EventEnd"].values[i]
                    label = tmp_report_event_df["EventClass"].values[i]
                    if label == "P":
                        continuous_report_event_df.loc[continuous_report_event_df["DateTime (UTC)"].between(cur_event_start - pd.Timedelta(seconds=1), cur_event_end + pd.Timedelta(seconds=1), inclusive="both"), "is_P"] = 1
                    elif label == "N":
                        continuous_report_event_df.loc[continuous_report_event_df["DateTime (UTC)"].between(cur_event_start - pd.Timedelta(seconds=1), cur_event_end + pd.Timedelta(seconds=1), inclusive="both"), "is_P"] = 0
                
                # plot the figure
                figure = plt.figure(figsize=(20, 10))
                
                plot_delta = int((end - start) / pd.Timedelta(seconds=1))
                shift_time = int(plot_delta * 0.05)

                # split the figure into 3 parts, the first part is true events, which contains 2 subplots, 
                # the second part is report events, which contains 2 subplots,
                # the third part is the classification result, which contains 1 subplot  

                # plot the true events              
                grid = plt.GridSpec(5, 10, hspace=0.2, wspace=0.2)
                ax1 = plt.subplot(grid[0:2, :])
                plt.plot(continuous_true_event_df["DateTime (UTC)"], continuous_true_event_df["is_P"], label="Stanford-defined", color="black")
                plt.xlim(start - pd.Timedelta(seconds=shift_time) , end + pd.Timedelta(seconds=shift_time))
                plt.setp(ax1.get_xticklabels(), visible=False)

                plt.yticks([0, 1])
                plt.ylabel("Released", fontsize=20, verticalalignment="center_baseline")
                plt.legend(loc=(0.9, 0.8))
                # plt.xlim(start - pd.Timedelta(shift_time, "s"), end + pd.Timedelta(shift_time, "s"))
                plt.title(f"Classification of {sensor_name} at " + datetime.datetime.strftime(date, "%Y-%m-%d"))

                # plot the report events
                ax2 = plt.subplot(grid[2:4, :])
                plt.plot(continuous_report_event_df["DateTime (UTC)"], continuous_report_event_df["is_P"], label="Team-defined", color="blue")
                plt.xlim(start - pd.Timedelta(seconds=shift_time) , end + pd.Timedelta(seconds=shift_time))
                plt.setp(ax2.get_xticklabels(), visible=False)
                plt.yticks([0, 1])
                # plt.xlim(start - pd.Timedelta(shift_time, "s"), end + pd.Timedelta(shift_time, "s"))


                plt.ylabel("Reported", fontsize=20, verticalalignment="center_baseline")
                plt.legend(loc=(0.9, 0.8))

                # plot the classification result
                # set the color map for the classification result

                colors = {
                        "TP": "#4daf4a",
                        "FP": "#e41a1c",
                        "FN": "#ff7f00",
                        "TN": "#377eb8", 
                        "NA": "#984ea3"
                    }

                # control the legend of the classification result
                nums = {
                        "TP": 0,
                        "FP": 0,
                        "FN": 0,
                        "TN": 0,
                        "NA": 0
                    }
                
                # set the label position for the classification result
                label_pos = {
                    "Stanford-defined": 2,
                    "Team-defined": 0,
                    "Time-based": -2
                }

                axes = plt.subplot(grid[4, :])

                # plot the Team-defined classification result
                if sensor_name != "Soofie":
                    for i in range(tmp_report_event_df.shape[0]):
                        cur_plot_start = max(tmp_report_event_df["EventStart"].values[i], start)
                        cur_plot_end = min(tmp_report_event_df["EventEnd"].values[i], end)
                        label = tmp_report_event_df["label"].values[i]

                        # if the classification result is the first one, then plot the label, which is the beautiful legend of the figure
                        if (label == "TP" and nums["TP"] == 0) or \
                            (label == "FP" and nums["FP"] == 0) or \
                                (label == "FN" and nums["FN"] == 0) or \
                                    (label == "TN" and nums["TN"] == 0) or \
                                        (label == str("NA") and nums["NA"] == 0):
                            plt.plot(
                                [cur_plot_start, cur_plot_end],
                                [label_pos["Team-defined"], label_pos["Team-defined"]],
                                lw = 15,
                                color = colors[label],
                                label = str("NA") if label == str("NA") else label
                            )
                            nums[label] += 1
                        else:
                            plt.plot(
                                [cur_plot_start, cur_plot_end],
                                [label_pos["Team-defined"], label_pos["Team-defined"]],
                                lw = 15,
                                color = colors[label]
                            )
                    

                # plot the Stanford-defined classification result
                if sensor_name != "Soofie":
                    for i in range(tmp_true_event_df.shape[0]):
                        cur_plot_start = max(tmp_true_event_df["EventStart"].values[i], start)
                        cur_plot_end = min(tmp_true_event_df["EventEnd"].values[i], end)
                        label = tmp_true_event_df["label"].values[i]

                        if (label == "TP" and nums["TP"] == 0) or\
                            (label == "FP" and nums["FP"] == 0) or\
                                (label == "FN" and nums["FN"] == 0) or\
                                    (label == "TN" and nums["TN"] == 0) or\
                                        (label == str("NA") and nums["NA"] == 0):
            
                            plt.plot(
                                [cur_plot_start, cur_plot_end],
                                [label_pos["Stanford-defined"], label_pos["Stanford-defined"]],
                                lw = 15,
                                color = colors[label],
                                label = str("NA") if label == str("NA") else label
                            )
                            nums[label] += 1
                        else:
                            plt.plot(
                                [cur_plot_start, cur_plot_end],
                                [label_pos["Stanford-defined"], label_pos["Stanford-defined"]],
                                lw = 15,
                                color = colors[label]
                            )

                # plot the Time-based classification result
                tmp_time_event_df = time_event_df[(time_event_df["Datetime (UTC)"] >= start) & (time_event_df["Datetime (UTC)"] <= end)].reset_index(drop=True)

                # find a segment which contains the same label and plot it

                time_start_idx = 0
                time_end_idx = 0
                while time_end_idx < tmp_time_event_df.shape[0]:

                    if tmp_time_event_df["norm_class"].values[time_start_idx] == tmp_time_event_df["norm_class"].values[time_end_idx]:
                        time_end_idx += 1
                        continue
                    else:
                        label = tmp_time_event_df["norm_class"].values[time_start_idx]
                        if label in colors.keys():
                            if (label == "TP" and nums["TP"] == 0) or\
                                  (label == "FP" and nums["FP"] == 0) or\
                                      (label == "FN" and nums["FN"] == 0) or\
                                          (label == "TN" and nums["TN"] == 0) or\
                                            (label == str("NA") and nums["NA"] == 0):
                                plt.plot(
                                    [tmp_time_event_df["Datetime (UTC)"].values[time_start_idx], tmp_time_event_df["Datetime (UTC)"].values[time_end_idx - 1]],
                                    [label_pos["Time-based"], label_pos["Time-based"]],
                                    lw = 15,
                                    color = colors[label],
                                    label = str("NA") if label == str("NA") else label
                                )
                                nums[label] += 1
                            else:
                                plt.plot(
                                    [tmp_time_event_df["Datetime (UTC)"].values[time_start_idx], tmp_time_event_df["Datetime (UTC)"].values[time_end_idx - 1]],
                                    [label_pos["Time-based"], label_pos["Time-based"]],
                                    lw = 15,
                                    color = colors[label]
                                )

                        else:
                            continue
                        time_start_idx = time_end_idx
                        time_end_idx += 1
                if time_start_idx < tmp_time_event_df.shape[0]:
                    label = tmp_time_event_df["norm_class"].values[time_start_idx]
                    if label in colors.keys():
                        if (label == "TP" and nums["TP"] == 0) or\
                                (label == "FP" and nums["FP"] == 0) or\
                                    (label == "FN" and nums["FN"] == 0) or\
                                        (label == "TN" and nums["TN"] == 0) or\
                                            (label == str("NA") and nums["NA"] == 0):
                                plt.plot(
                                    [tmp_time_event_df["Datetime (UTC)"].values[time_start_idx], tmp_time_event_df["Datetime (UTC)"].values[time_end_idx - 1]],
                                    [label_pos["Time-based"], label_pos["Time-based"]],
                                    lw = 15,
                                    color = colors[tmp_time_event_df["norm_class"].values[time_start_idx]],
                                    label = str("NA") if label == str("NA") else label
                                )
                                nums[label] += 1
                        else:
                            plt.plot(
                                [tmp_time_event_df["Datetime (UTC)"].values[time_start_idx], tmp_time_event_df["Datetime (UTC)"].values[time_end_idx - 1]],
                                [label_pos["Time-based"], label_pos["Time-based"]],
                                lw = 15,
                                color = colors[tmp_time_event_df["norm_class"].values[time_start_idx]]
                            )
                    else:
                        continue

                axes.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M')) 
                plt.setp(axes.get_yticklabels(), visible=False)
                plt.xlabel("Time (UTC)")
                l1 = "Stanford-defined"
                l2 = "     Team-defined"
                l3 = "        Time-based"
                l_len = max(len(l1), len(l2), len(l3))
                title = f"{l1}"  + "\n" + f"{l2}" + "\n" + f"{l3}"
                plt.ylabel(title, rotation="horizontal", labelpad=90, verticalalignment="center_baseline")
                plt.ylim(-5, 5)
                plt.xlim(start - pd.Timedelta(seconds=shift_time), end + pd.Timedelta(seconds=shift_time))
                plt.legend(loc=(1, 0))
                plt.savefig(PurePath(plot_save_dir, datetime.datetime.strftime(date, "%Y-%m-%d") + ".jpg"), dpi=300)
                plt.close()


plot_sensors = sensor_names
save_dir = PurePath("../../results/03_DetectionAnalysis/Test-case Matching Data/Matching Figures Per Day/")
prev_str = ""
if short_stack:
    prev_str += "ss_"
save_dir = PurePath(save_dir, prev_str + "{}xradius_{}xseconds/".format(threshold, ignore_duration))
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
# plot_sensors = ["Andium"]
print("Plotting compariso figures...")
plot_comparison_figuires(stanford_events_dict, team_events_dict, time_events_dict, plot_sensors, save_dir)

Plotting compariso figures...


Andium    :   0%|          | 0/52 [00:00<?, ?it/s]

2022-10-10 00:00:00 Andium
14945
9
5


Andium    :   4%|▍         | 2/52 [00:01<00:32,  1.52it/s]

2022-10-11 00:00:00 Andium
73882
11
8


...

Sensirion :  98%|█████████▊| 51/52 [00:44<00:01,  1.00s/it]

2022-11-29 00:00:00 Sensirion
86161
6
3


Sensirion : 100%|██████████| 52/52 [00:46<00:00,  1.17s/it]

2022-11-30 00:00:00 Sensirion
67441
3
3


Sensirion : 100%|██████████| 52/52 [00:47<00:00,  1.10it/s]
